In [1]:
# imports

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np

import matplotlib.pyplot as plt

In [6]:
# import load_data function from loadData.py
from loadData import load_data

# data = load_data("Hello There (Obi Wan) - Sound Effect (HD).wav")
data = load_data("sine_wave.wav")
print(data[10])


91


In [7]:
# splits data into random batches
def batch(set):
    batch_size = 5
    batched_set = []
    # choose batch_size random numbers between 0 and len(set)
    random_indices = np.random.randint(0, len(set), batch_size)
    for i in random_indices:
        batched_set.append(set[i])
    return batched_set

batched_data = batch(data)
print(fourier(batched_data,3))

NameError: name 'fourier' is not defined

In [ ]:
# define the network class

order = 5
dims = 2*order+1

class Net(nn.Module):
    def __init__(self, dims):
        super().__init__()
        self.dims = dims
        self.fc1 = nn.Linear(self.dims, 2*self.dims)
        self.fc2 = nn.Linear(2*self.dims, 2*self.dims)
        self.fc3 = nn.Linear(2*self.dims, 2*self.dims)
        self.fc4 = nn.Linear(2*self.dims, 2*self.dims)
        self.fc5 = nn.Linear(2*self.dims, 1)


    def forward(self, x):
        x = torch.flatten(x, -1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.fc5(x)
        return x

In [ ]:
model = Net(dims).double()
test_input = fourier([100,200],order)
print(model.forward(test_input))

In [ ]:
def test(model, test_loader):
    correct = 0
    total = 0
    model.eval()

    with torch.no_grad():
        for data, labels in test_loader:
            output = model(data)
            predicted = output.data
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    accuracy = correct / total
    print(f"Accuracy: {accuracy * 100}%")
    return accuracy


In [ ]:
# Format data into dataset type
class FourierDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        if self.transform:
            sample = self.transform(sample)
        return sample

train_dataset = FourierDataset(batched_data)
test_dataset = FourierDataset(batched_data)


# data loader
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=True)


In [ ]:
# create an instance of the network
model = Net(dims).double()

# define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# run one forward pass on the first item in the train_loader
# print(train_labels[0].item())
train_data_iter = iter(train_loader)
train_data = next(train_data_iter)


In [ ]:
def train(model, train_loader, criterion, optimizer, epochs):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs = data.double()
            labels = data
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}")
    print('Finished Training')


In [ ]:
# train the network
train(model, train_loader, criterion, optimizer, 1000)

# test the network
test(model, test_loader)

In [ ]:
print(test_labels[0].item())
model.forward(test_data[0])
print(torch.argmax(model.forward(test_data[0])).item())


In [ ]:
# save the model
torch.save(model.state_dict(), "model.pt")

In [ ]:
# load the model if needed
model = Net()
model.load_state_dict(torch.load("model.pt"))
model.eval()

In [ ]:
import csv

user_input = []
with open('user_input.csv', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        for i in range(0, len(row)):
            row[i] = float(row[i])/255
        user_input.append(row)

user_input = torch.tensor(user_input)

prediction = model.forward(user_input[0])

# print prediction as dictionary of index: value to 2sf (rounded)
prediction_dict = {}
for i in range(0, len(prediction)):
    prediction_dict[i] = round(prediction[i].item(), 2)
print(prediction_dict)

# bar chart of prediction
plt.bar(range(len(prediction_dict)), list(prediction_dict.values()), align='center')


# print index of max value
print(torch.argmax(prediction).item())
